In [1]:
!pip install sentence-transformers torch pandas scikit-learn


In [2]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import torch

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

data = pd.read_csv('/content/Job Dataset.csv',encoding='ISO-8859-1')
train_examples = []
unique_skills = set()

# Preprocessing function
def preprocess_data(data):
    data = data.dropna(subset=['Job Title', 'Job Skills'])
    data['Job Title'] = data['Job Title'].str.lower()
    data['Job Skills'] = data['Job Skills'].str.lower()
    data['skills_list'] = data['Job Skills'].apply(lambda x: x.split(","))

    for _, row in data.iterrows():
        job_title = row['Job Title']
        for skill in row['skills_list']:
            skill = skill.strip()
            train_examples.append(InputExample(texts=[job_title, skill], label=1))
            unique_skills.add(skill)

        unused_skills = list(unique_skills - set(row['skills_list']))
        if unused_skills:
            for _ in range(len(row['skills_list'])):
                random_skill = np.random.choice(unused_skills)
                train_examples.append(InputExample(texts=[job_title, random_skill], label=0))

preprocess_data(data)

# Define DataLoader and Loss
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=8)
train_loss = losses.CosineSimilarityLoss(model)

# Training parameters
epochs = 4
patience = 1
min_delta = 1e-4
best_loss = float('inf')
no_improvement_epochs = 0

# Training loop
for epoch in range(epochs):
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              epochs=1,
              warmup_steps=100,
              weight_decay=0.01,
              show_progress_bar=True)

    # Calculate validation loss
    model.eval()


# Encode unique skills
skill_embeddings = model.encode(list(unique_skills), convert_to_tensor=True)

# Skill prediction function
def predict_skills(job_title, top_n=10):
    job_embedding = model.encode(job_title, convert_to_tensor=True).cpu()
    similarities = cosine_similarity(job_embedding.unsqueeze(0).detach().numpy(), skill_embeddings.detach().numpy())
    top_indices = np.argsort(similarities[0])[::-1][:top_n]
    top_skills = [list(unique_skills)[i] for i in top_indices]
    return ', '.join(top_skills)

new_job_title = "Web developer"
predicted_skills = predict_skills(new_job_title)
print(f"Predicted skills for '{new_job_title}': {predicted_skills}")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
<ipython-input-7-3482ec93aa83>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Job Title'] = data['Job Title'].str.lower()
<ipython-input-7-3482ec93aa83>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

Step,Training Loss
500,0.213200
1000,0.186300
1500,0.181500
2000,0.170200


Step,Training Loss
500,0.142900
1000,0.141500
1500,0.151400
2000,0.152800


Step,Training Loss
500,0.101500
1000,0.104500
1500,0.125700
2000,0.141200


Step,Training Loss
500,0.064700
1000,0.070100
1500,0.098300
2000,0.130300


Predicted skills for 'Web developer': html, microservices, css, php, restful apis, angular, javascript, apex, html), typescript


In [10]:
def predict_skills(job_title, top_n=10):
    job_embedding = model.encode(job_title, convert_to_tensor=True)
    similarities = cosine_similarity(job_embedding.unsqueeze(0).detach().numpy(), skill_embeddings.detach().numpy())
    top_indices = np.argsort(similarities[0])[::-1][:top_n]
    top_skills = [list(unique_skills)[i] for i in top_indices]
    return ', '.join(top_skills)

new_job_title = "Web developer"
predicted_skills = predict_skills(new_job_title)
print(f"Predicted skills for '{new_job_title}': {predicted_skills}")


Predicted skills for 'Web developer': html, microservices, css, php, restful apis, angular, javascript, apex, html), typescript


In [11]:
import re

def predict_skills(job_title, top_n=10):
    job_embedding = model.encode(job_title, convert_to_tensor=True)
    similarities = cosine_similarity(job_embedding.unsqueeze(0).detach().numpy(), skill_embeddings.detach().numpy())
    top_indices = np.argsort(similarities[0])[::-1][:top_n]
    # Remove unwanted characters and join the skills
    top_skills = [re.sub(r"[^\w\s]", "", list(unique_skills)[i].strip()) for i in top_indices]
    return ', '.join(top_skills)

new_job_title = "Web developer"
predicted_skills = predict_skills(new_job_title)
print(f"Predicted skills for '{new_job_title}': {predicted_skills}")


Predicted skills for 'Web developer': html, microservices, css, php, restful apis, angular, javascript, apex, html, typescript


In [12]:
new_job_title = "content writer"
predicted_skills = predict_skills(new_job_title)
print(f"Predicted skills for '{new_job_title}': {predicted_skills}")

Predicted skills for 'content writer': content writing, content publishing, text content creation, content creation, technical content creation, content management, content management systems, content strategy, blog writing, content editing
